In [1]:
import arcpy
import requests
import os
import zipfile
import io

#Assign working directory

In [2]:
workspace = r'C:\Users\cason\Desktop\GIS5571\Lab2\part2'

#Assign variables for data

In [3]:
mn_counties = r'C:\Users\cason\Desktop\GIS5571\Lab2\part2\data\mn_county_boundaries.shp'
mn_dem = r'C:\Users\cason\Desktop\GIS5571\Lab2\part2\data\elev_30m_digital_elevation_model.gdb\digital_elevation_model_30m'
mn_nlcd = r'C:\Users\cason\Desktop\GIS5571\Lab2\part2\data\NLCD_2019_Land_Cover.tif'
mn_streams = r'C:\Users\cason\Desktop\GIS5571\Lab2\part2\data\streams_with_strahler_stream_order.shp'
mn_roads = r'C:\Users\cason\Desktop\GIS5571\Lab2\part2\data\NLCD_2019_Land_Cover_Impervious_Descriptor.tif'

#Create study extent area from Minnesota Counties shapefile

In [4]:
# Select Wabasha, Winona, and Olmsted counties from mn_counties feature class
mn_county_selection = arcpy.conversion.FeatureClassToFeatureClass(mn_counties, 
                                                                  workspace, 
                                                                  "mn_counties_study_extent", 
                                                                  "CTY_NAME = 'Wabasha' Or CTY_NAME = 'Winona' Or CTY_NAME = 'Olmsted'", 
                                                                  r'AREA "AREA" true true false 19 Double 0 0,First,#,C:\Users\cason\Desktop\GIS5571\Lab2\part2\data\mn_county_boundaries.shp,AREA,-1,-1;PERIMETER "PERIMETER" true true false 19 Double 0 0,First,#,C:\Users\cason\Desktop\GIS5571\Lab2\part2\data\mn_county_boundaries.shp,PERIMETER,-1,-1;CTYONLY_ "CTYONLY_" true true false 19 Double 0 0,First,#,C:\Users\cason\Desktop\GIS5571\Lab2\part2\data\mn_county_boundaries.shp,CTYONLY_,-1,-1;CTYONLY_ID "CTYONLY_ID" true true false 19 Double 0 0,First,#,C:\Users\cason\Desktop\GIS5571\Lab2\part2\data\mn_county_boundaries.shp,CTYONLY_ID,-1,-1;COUN "COUN" true true false 4 Short 0 4,First,#,C:\Users\cason\Desktop\GIS5571\Lab2\part2\data\mn_county_boundaries.shp,COUN,-1,-1;CTY_NAME "CTY_NAME" true true false 20 Text 0 0,First,#,C:\Users\cason\Desktop\GIS5571\Lab2\part2\data\mn_county_boundaries.shp,CTY_NAME,0,20;CTY_ABBR "CTY_ABBR" true true false 4 Text 0 0,First,#,C:\Users\cason\Desktop\GIS5571\Lab2\part2\data\mn_county_boundaries.shp,CTY_ABBR,0,4;CTY_FIPS "CTY_FIPS" true true false 4 Short 0 4,First,#,C:\Users\cason\Desktop\GIS5571\Lab2\part2\data\mn_county_boundaries.shp,CTY_FIPS,-1,-1;Shape_Leng "Shape_Leng" true true false 19 Double 0 0,First,#,C:\Users\cason\Desktop\GIS5571\Lab2\part2\data\mn_county_boundaries.shp,Shape_Leng,-1,-1;Shape_Area "Shape_Area" true true false 19 Double 0 0,First,#,C:\Users\cason\Desktop\GIS5571\Lab2\part2\data\mn_county_boundaries.shp,Shape_Area,-1,-1', '')

In [5]:
# Dissolve selected counties to create study area extent boundary
arcpy.management.Dissolve(mn_county_selection, 
                          r'C:\Users\cason\Desktop\GIS5571\Lab2\part2\study_extent', 
                          None, None, "MULTI_PART", "DISSOLVE_LINES", '')

<Result 'C:\\Users\\cason\\Desktop\\GIS5571\\Lab2\\part2\\study_extent.shp'>

In [6]:
#Create variable for study extent area boundary
study_extent = r'C:\Users\cason\Desktop\GIS5571\Lab2\part2\study_extent'

#Roads: Extract raster by study extent mask and reclassify

In [8]:
#Use study extent as a mask for extracting roads raster
roads_study_extent = arcpy.sa.ExtractByMask(mn_roads, 
                                            study_extent, 
                                            "INSIDE", 
                                            '524966.6376 4853462.8394 637916.1448 4922619.9426 PROJCS["NAD_1983_UTM_Zone_15N",GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["False_Easting",500000.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",-93.0],PARAMETER["Scale_Factor",0.9996],PARAMETER["Latitude_Of_Origin",0.0],UNIT["Meter",1.0]]'); 
roads_study_extent.save(r"C:\Users\cason\Documents\ArcGIS\Projects\Lab1\Lab2_Notes_10.19\Default.gdb\Extract_NLCD1")

In [9]:
#Reclassify masked roads raster. Unclassified: 10, Primary: 1, Secondary: 1, Tertiary: 1, Non-road/non-energy impervious: 2, LCMAP impervious: 3, Wind turbines: 7
roads_reclass = arcpy.ddd.Reclassify(roads_study_extent, 
                                     "Class_Name", 
                                     "Unclassified 10;'Primary road' 1;'Secondary road' 1;'Tertiary road' 1;'Non-road non-energy impervious' 2;'LCMAP impervious' 3;'Wind turbines' 7", r"C:\Users\cason\Desktop\GIS5571\Lab2\part2\data\roads_reclass", "DATA")

#Slope: Extract raster by study extent, derive slope from DEM, and reclassify

In [10]:
#Use study extent as a mask for extracting DEM raster
dem_study_extent = arcpy.sa.ExtractByMask(mn_dem, 
                                          study_extent, 
                                          "INSIDE", 
                                          '524966.6376 4853462.8394 637916.1448 4922619.9426 PROJCS["NAD_1983_UTM_Zone_15N",GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["False_Easting",500000.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",-93.0],PARAMETER["Scale_Factor",0.9996],PARAMETER["Latitude_Of_Origin",0.0],UNIT["Meter",1.0]]'); 
dem_study_extent.save(r"C:\Users\cason\Desktop\GIS5571\Lab2\part2\data\dem_se")

In [11]:
#Use slope tool on masked DEM raster and use percent rise as output measurement
slope = arcpy.sa.Slope(dem_study_extent, 
                       "PERCENT_RISE", 
                       1, 
                       "PLANAR", 
                       "METER"); 
slope.save(r"C:\Users\cason\Desktop\GIS5571\Lab2\part2\data\slope")

In [13]:
#Reclassify masked slope raster. 0-3% = 1, 3-6% = 2, 6-12% = 3, 12-15% = 4, 15-18% = 8, 18-21% = 9, >21% = 10
slope_reclass = arcpy.sa.Reclassify(slope, 
                                    "VALUE", 
                                    "0 3 1;3 6 2;6 12 3;12 15 4;15 18 8;18 21 9;21 24 10;24 27 10;27 30 10;30 533.479797 10", 
                                    "DATA"); 
slope_reclass.save(r"C:\Users\cason\Desktop\GIS5571\Lab2\part2\data\slope_reclass")

#Land Cover: Extract raster by study extent and reclassify

In [14]:
#Use study extent as a mask for extracting NLCD raster
nlcd_study_extent = arcpy.sa.ExtractByMask(mn_nlcd, 
                                           study_extent, 
                                           "INSIDE", 
                                           '524966.6376 4853462.8394 637916.1448 4922619.9426 PROJCS["NAD_1983_UTM_Zone_15N",GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["False_Easting",500000.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",-93.0],PARAMETER["Scale_Factor",0.9996],PARAMETER["Latitude_Of_Origin",0.0],UNIT["Meter",1.0]]'); 
nlcd_study_extent.save(r"C:\Users\cason\Desktop\GIS5571\Lab2\part2\data\nlcd_se")

In [15]:
#Reclassify masked NLCD raster. Open Water = 10, Developed Open Space = 2, Developed Low Intensity = 2, Developed Medium Intensity = 2, Developed High Intensity = 2, Barren Land = 5, Deciduous Forest = 7, Evergreen Forest = 7, Mixed Forest = 7, Shrub = 7, Herbaceous = 7, Hay/Pasture = 9, Cultivated Crops = 9, Woody Wetlands = 9, Emergent Herbaceous Wetlands = 10
nlcd_reclass = arcpy.sa.Reclassify(nlcd_study_extent, 
                                   "NLCD_LAND", 
                                   "'Open Water' 10;'Developed, Open Space' 2;'Developed, Low Intensity' 2;'Developed, Medium Intensity' 2;'Developed, High Intensity' 2;'Barren Land' 5;'Deciduous Forest' 7;'Evergreen Forest' 7;'Mixed Forest' 7;Shrub/Scrub 7;Herbaceous 7;Hay/Pasture 9;'Cultivated Crops' 9;'Woody Wetlands' 9;'Emergent Herbaceous Wetlands' 10", 
                                   "DATA"); 
nlcd_reclass.save(r"C:\Users\cason\Desktop\GIS5571\Lab2\part2\data\nlcd_reclass")

#Streams: Clip feature class and reclassify

In [16]:
#Clip mn_streams shapefile within study extent
streams_clip = arcpy.analysis.Clip(mn_streams, 
                                   study_extent, 
                                   r"C:\Users\cason\Desktop\GIS5571\Lab2\part2\data\streams_clip", 
                                   None)

In [17]:
#Convert streams shapefile to raster
streams_raster = arcpy.conversion.FeatureToRaster(streams_clip, 
                                                  "SO_VALUE", 
                                                  r"C:\Users\cason\Desktop\GIS5571\Lab2\part2\data\stream_raster", 
                                                  r"C:\Users\cason\Desktop\GIS5571\Lab2\part2\data\dem_se")

In [18]:
#Reclassify streams raster. Smaller streams are classified lower than larger streams.
streams_reclass = arcpy.sa.Reclassify(streams_raster, 
                                      "VALUE", 
                                      "1 1;2 2;3 8;4 8;5 10;6 10;8 10", 
                                      "NODATA"); 
streams_reclass.save(r"C:\Users\cason\Desktop\GIS5571\Lab2\part2\data\streams_rc")

In [25]:
#Performed Raster Calculator outside of script, because I continued to get errors, and filled in all non-stream areas with 0 value cells
final_streams_reclass = r'C:\Users\cason\Desktop\GIS5571\Lab2\part2\data\final_stream'

#Perform Weighted Overlay to generate a Cost Surface

In [2]:
cost_surface = arcpy.sa.WeightedOverlay(r"('C:\Users\cason\Desktop\GIS5571\Lab2\part2\data\slope_reclass' 25 'VALUE' (1 1; 2 2; 3 3; 4 4; 8 8; 9 9; 10 10; NODATA NODATA); 'C:\Users\cason\Desktop\GIS5571\Lab2\part2\data\roads_reclass' 25 'VALUE' (1 1; 2 2; 3 3; 7 7; 10 10; NODATA NODATA); 'C:\Users\cason\Desktop\GIS5571\Lab2\part2\data\nlcd_reclass' 25 'VALUE' (2 2; 5 5; 7 7; 9 9; 10 10; NODATA NODATA); 'C:\Users\cason\Desktop\GIS5571\Lab2\part2\data\final_stream' 25 'VALUE' (0 1; 1 1; 2 2; 8 8; 10 10; NODATA NODATA));1 10 1"); 
cost_surface.save(r"C:\Users\cason\Desktop\GIS5571\Lab2\part2\data\cost_surface")

#Create variables for the location of Dory's farm and Dory's destination

In [3]:
#Create variables for CSV files created in Excel
start_csv = r'C:\Users\cason\Desktop\GIS5571\Lab2\part2\start.csv'
end_csv = r'C:\Users\cason\Desktop\GIS5571\Lab2\part2\end.csv'

In [7]:
#Create start point shapefile from CSV file
start = arcpy.management.XYTableToPoint(start_csv, 
                                        r"C:\Users\cason\Desktop\GIS5571\Lab2\part2\data\start_point.shp", 
                                        "longitude", 
                                        "latitude", 
                                        None, 
                                        'GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]];-400 -400 1000000000;-100000 10000;-100000 10000;8.98315284119521E-09;0.001;0.001;IsHighPrecision')

In [8]:
#Create end point shapefile from CSV file
end = arcpy.management.XYTableToPoint(end_csv, 
                                      r"C:\Users\cason\Desktop\GIS5571\Lab2\part2\data\end_point.shp", 
                                      "longitude", 
                                      "latitude", 
                                      None, 
                                      'GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]];-400 -400 1000000000;-100000 10000;-100000 10000;8.98315284119521E-09;0.001;0.001;IsHighPrecision')

#Perform Cost Distance analysis

In [11]:
cost_distance = arcpy.sa.CostDistance("start_point", 
                                      "cost_surface", 
                                      None, 
                                      r"C:\Users\cason\Desktop\GIS5571\Lab2\part2\data\direct_surf", 
                                      None, None, None, None, ''); 
cost_distance.save(r"C:\Users\cason\Desktop\GIS5571\Lab2\part2\data\cost_dist_fin")

In [12]:
direction_surface = r'C:\Users\cason\Desktop\GIS5571\Lab2\part2\data\direct_surf'

#Perform Cost Path analysis

In [14]:
cost_path = arcpy.sa.CostPath(end, 
                               cost_distance, 
                               direction_surface, 
                               "BEST_SINGLE", 
                               "FID", 
                               "INPUT_RANGE"); 
cost_path.save(r"C:\Users\cason\Desktop\GIS5571\Lab2\part2\data\optimal_route")